<a href="https://colab.research.google.com/github/Bibek04-bit/Flood-Forecasting/blob/main/Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from sklearn.model_selection import train_test_split

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
path_to_file = '/content/Flood_data_linear_reg.csv'
df = pd.read_csv(path_to_file)
print(df.head())

         Date  Daily flow Rate  Gauge height flow      NDVI  sm_top_0_10cm  \
0  2010-01-01              401               3.19  0.806800         12.728   
1  2010-01-02              397               3.18  0.805588         12.628   
2  2010-01-03              395               3.17  0.804375         12.533   
3  2010-01-04              390               3.16  0.803163         12.444   
4  2010-01-05              392               3.17  0.801950         12.356   

   sm_10_40cm  24h accumulated  Daily Max Temp  Daily Min Temp  Flood_Level  
0   41.205952              0.1            14.0             8.0            0  
1   40.953522              1.8            14.4            10.8            0  
2   40.707092              1.6            18.0            10.4            0  
3   40.467663              0.0            19.5             9.5            0  
4   40.249233              0.0            20.5            10.0            0  


In [9]:
df = df.dropna()
if 'Date' in df.columns:
  df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               3652 non-null   datetime64[ns]
 1   Daily flow Rate    3652 non-null   int64         
 2   Gauge height flow  3652 non-null   float64       
 3   NDVI               3652 non-null   float64       
 4   sm_top_0_10cm      3652 non-null   float64       
 5   sm_10_40cm         3652 non-null   float64       
 6   24h accumulated    3652 non-null   float64       
 7   Daily Max Temp     3652 non-null   float64       
 8   Daily Min Temp     3652 non-null   float64       
 9   Flood_Level        3652 non-null   int64         
dtypes: datetime64[ns](1), float64(7), int64(2)
memory usage: 285.4 KB
